# Import Libraries

In [ ]:
from yahoo_fin.stock_info import *

import pandas as pd
import numpy as np
from numpy import mean, std

import datetime
import time
import os, sys
import shutil

from tqdm import tqdm #Used in the for loops to track the progress of the loop

# Initial Setup

In [ ]:
#Create a Data Folder
Data_folder = os.path.abspath(os.getcwd() +'/Data/')
if not os.path.exists(Data_folder):
    os.makedirs(Data_folder)

#Clean older files and folders in the Data Folder
filelist = [ f for f in os.listdir(Data_folder)]
for f in filelist:
    shutil.rmtree(os.path.join(Data_folder, f), ignore_errors=True)

filelist = [ f for f in os.listdir(Data_folder)]
for f in filelist:
    os.remove(os.path.join(Data_folder, f))

#Create New Folder for Stock Indices
stock_indices_folder = os.path.join(Data_folder, 'Stock Indices')
if not os.path.exists(stock_indices_folder):
    os.makedirs(stock_indices_folder)
    
#Create New Folder for FX Rates
FX_rates_folder = os.path.join(Data_folder, 'FX Rates')
if not os.path.exists(FX_rates_folder):
    os.makedirs(FX_rates_folder)

## Define the Indices and Currencies

In [ ]:
#Instantiate the Indices data
indices_data = pd.read_csv('Stock_Indices.csv') 

#Create Indices variables and Manage the Column Data to create new variables
indices_tickers = indices_data['Yahoo Symbol']

FX_rates_symbols = indices_data['Yahoo Currency Symbol']
FX_rates_symbols = FX_rates_symbols.drop_duplicates().dropna()

# Dowload Data

## Data for Indices Tickers

In [ ]:
#Price Data: Defining the Start and End dates 
finishing_period = time.strftime('%d/%m/%Y')

    ##Let's define the start date as 1 year ago
beginning_period_unformat = datetime.datetime.strptime(finishing_period, '%d/%m/%Y') - datetime.timedelta(days=370)
beginning_period = beginning_period_unformat.strftime('%m/%d/%Y')

for ticker in tqdm(indices_tickers):
    try:

        historical_data = get_data(ticker, start_date = beginning_period , end_date = finishing_period)

        #Let's Create a DataFrame with the downloaded Data
        df = pd.DataFrame(historical_data, columns=['open', 'high', 'low', 'close', 'adjclose', 'volume'])
        df['date'] = pd.to_datetime(df.index, format='%Y/%m/%d')

        close_price = df.close.values

        #Construct Ticker Table to Print
        index_data_table = pd.DataFrame(data={'date': df['date'], ticker: df['close']}).set_index('date')

        index_data_table = index_data_table.reset_index()
        index_data_table.sort_values(by='date', inplace=True, ascending=False)
        index_data_table = index_data_table.set_index('date')
        index_data_table.to_csv(os.path.join(stock_indices_folder,ticker)+'_Table.csv')
    except:
        continue

In [ ]:
#Join All the Ticker Data Files into a Table
files= os.listdir(stock_indices_folder)

indices_data_table = pd.read_csv(os.path.join(stock_indices_folder,files[0]))

for file in files[1:]:
    df = pd.read_csv(os.path.join(stock_indices_folder,file)) 
    indices_data_table = indices_data_table.merge(df, on='date', how='left')

#Rearrange the DataFrame format: fill na's, rounding, etc.
indices_data_table = indices_data_table.fillna(method='bfill')
indices_data_table = indices_data_table.round(2)
indices_data_table = indices_data_table.set_index('date')
indices_data_table = indices_data_table.head(-1)

indices_data_table.to_csv(os.path.join(Data_folder,'Stock_Indices_Data_Table.csv'))

## Data for Currencies

In [ ]:
#Price Data: Defining the Start and End dates 
finishing_period = time.strftime('%d/%m/%Y')

    ##Let's define the start date as 1 year ago
beginning_period_unformat = datetime.datetime.strptime(finishing_period, '%d/%m/%Y') - datetime.timedelta(days=370)
beginning_period = beginning_period_unformat.strftime('%m/%d/%Y')

for symbol in tqdm(FX_rates_symbols):
    try:

        historical_data = get_data(symbol, start_date = beginning_period , end_date = finishing_period)

        #Let's Create a DataFrame with the downloaded Data
        df = pd.DataFrame(historical_data, columns=['open', 'high', 'low', 'close', 'adjclose', 'volume'])
        df['date'] = pd.to_datetime(df.index, format='%Y/%m/%d')

        close_price = df.close.values
        
        symbol = symbol[:6]
        #Construct Ticker Table to Print
        currency_data_table = pd.DataFrame(data={'date': df['date'], symbol: df['close']}).set_index('date')

        currency_data_table = currency_data_table.reset_index()
        currency_data_table.sort_values(by='date', inplace=True, ascending=False)
        currency_data_table = currency_data_table.set_index('date')
        currency_data_table.to_csv(os.path.join(FX_rates_folder,symbol)+'_Table.csv')
    except:
        continue

In [ ]:
#Join All the Currencies Data Files into a Table
files= os.listdir(FX_rates_folder)

currencies_data_table = pd.read_csv(os.path.join(FX_rates_folder,files[0]))

for file in files[1:]:
    df = pd.read_csv(os.path.join(FX_rates_folder,file)) 
    currencies_data_table = currencies_data_table.merge(df, on='date', how='left')

#Rearrange the DataFrame format: fill na's, rounding, etc.
currencies_data_table = currencies_data_table.fillna(method='bfill')
currencies_data_table = currencies_data_table.set_index('date')
currencies_data_table = currencies_data_table.head(-1)

currencies_data_table.to_csv(os.path.join(Data_folder,'FX_rates_Data_Table.csv'))

## Create the Table Indices Tickers based on same Currency

In [ ]:
#Merge Indices data with Currencies Data
indices_currencies_data_table = indices_data_table.merge(currencies_data_table,on='date', how='left')
indices_currencies_data_table = indices_currencies_data_table.fillna(method='bfill')
indices_currencies_data_table['USDUSD'] = 1

In [ ]:
#Convert Indices' values to same base Currency (USD) for proper comparison analysis
for ticker in indices_tickers:
    if indices_data[indices_data['Yahoo Symbol'] == ticker]['Yahoo Currency Symbol'].isnull().iloc[0]:
        currency = 'USDUSD'
    else:
        currency = indices_data[indices_data['Yahoo Symbol'] == ticker]['Yahoo Currency Symbol'].iloc[0][:6]
    indices_currencies_data_table[ticker] = indices_currencies_data_table[ticker]*indices_currencies_data_table[currency]

In [ ]:
#Limit the DataFrame to include just the Stock Indices (converted)
currencies = []
for currency in FX_rates_symbols:
    currency = currency[:6]
    currencies.append(currency)

indices_currencies_data_table = indices_currencies_data_table.drop(columns = currencies).drop(columns = ['USDUSD'])

# Calculate the Metrics used in RRG

## Calculate JdK- RS

In [ ]:
indices_currencies_data_table.sort_values(by='date', inplace=True, ascending=True)

#Calculate the 1-day Returns for the Indices
indices_currencies_data_table = indices_currencies_data_table.pct_change(1)

#Calculate the Indices' value on and Index-Base (100) considering the calculated returns
indices_currencies_data_table.iloc[0] = 100
for ticker in indices_currencies_data_table.columns:
    for i in range(1, len(indices_currencies_data_table[ticker])):
        indices_currencies_data_table[ticker][i] = indices_currencies_data_table[ticker][i-1]*(1+indices_currencies_data_table[ticker][i])

In [ ]:
#Define the Index for comparison (Benchamrk Index): FTSE Global Equity Index Series
bechmark = 'VWRL.AS'
benchmark_values = indices_currencies_data_table[bechmark]

indices_currencies_data_table = indices_currencies_data_table.drop(columns = bechmark)

#Calculate the relative Performance of the Index in relation to the Benchmark
for ticker in indices_currencies_data_table.columns:   
    indices_currencies_data_table[ticker] = indices_currencies_data_table[ticker]/benchmark_values - 1

#Normalize the Values considering a 14-days Window (Note: 10 weekdays)
for ticker in indices_currencies_data_table.columns: 
    indices_currencies_data_table[ticker] = 100 + ((indices_currencies_data_table[ticker] - indices_currencies_data_table[ticker].rolling(10).mean())/indices_currencies_data_table[ticker].rolling(10).std() + 1)
    
# Rouding and Exclusing NA's
indices_currencies_data_table = indices_currencies_data_table.round(2).dropna()
JDK_RS_ratio = indices_currencies_data_table

## Calculate JdK- RS

In [ ]:
#Calculate the Momentum of the RS-ratio
JDK_RS_momentum = JDK_RS_ratio.pct_change(10)

#Normalize the Values considering a 14-days Window (Note: 10 weekdays)
for ticker in JDK_RS_momentum.columns: 
    JDK_RS_momentum[ticker] = 100 + ((JDK_RS_momentum[ticker] - JDK_RS_momentum[ticker].rolling(10).mean())/JDK_RS_momentum[ticker].rolling(10).std() + 1)

# Rounding and Excluding NA's
JDK_RS_momentum = JDK_RS_momentum.round(2).dropna()

In [ ]:
#Adjust DataFrames to be shown in Monthly terms
JDK_RS_ratio = JDK_RS_ratio.reset_index()
JDK_RS_ratio['date'] = pd.to_datetime(JDK_RS_ratio['date'], format='%Y-%m-%d')
JDK_RS_ratio = JDK_RS_ratio.set_index('date')
JDK_RS_ratio = JDK_RS_ratio.resample('M').ffill()

#... now for JDK_RS Momentum
JDK_RS_momentum = JDK_RS_momentum.reset_index()
JDK_RS_momentum['date'] = pd.to_datetime(JDK_RS_momentum['date'], format='%Y-%m-%d')
JDK_RS_momentum = JDK_RS_momentum.set_index('date')
JDK_RS_momentum = JDK_RS_momentum.resample('M').ffill()

# Create the DataFrames for Creating the ScaterPlots

## DataFrame for ScaterPlot showing multiple Countries

In [ ]:
# Create a Dictionary for renaming Columns
columns_rename = dict(zip(indices_data['Yahoo Symbol'],indices_data['Symbol']))

#Rename Columns according to the Symbols and not Yahoo Symbols
JDK_RS_ratio = JDK_RS_ratio.rename(columns = columns_rename)
JDK_RS_momentum = JDK_RS_momentum.rename(columns = columns_rename)

In [ ]:
#Create a Sub-Header to the DataFrame: 'JDK_RS_ratio' -> As later both RS_ratio and RS_momentum will be joint
JDK_RS_ratio_subheader = pd.DataFrame(np.zeros((1,JDK_RS_ratio.columns.shape[0])),columns=JDK_RS_ratio.columns)
JDK_RS_ratio_subheader.iloc[0] = 'JDK_RS_ratio'

JDK_RS_ratio_total = pd.concat([JDK_RS_ratio_subheader, JDK_RS_ratio], axis=0)

#... same for JDK_RS Momentum
JDK_RS_momentum_subheader = pd.DataFrame(np.zeros((1,JDK_RS_momentum.columns.shape[0])),columns=JDK_RS_momentum.columns)
JDK_RS_momentum_subheader.iloc[0] = 'JDK_RS_momentum'

JDK_RS_momentum_total = pd.concat([JDK_RS_momentum_subheader, JDK_RS_momentum], axis=0)

In [ ]:
#Join both DataFrames
RRG_df = pd.concat([JDK_RS_ratio_total, JDK_RS_momentum_total], axis=1, sort=True)
RRG_df = RRG_df.sort_index(axis=1)

# Creating the Scatter Plots

In [ ]:
#Libraries for the Plotting
from bokeh.plotting import figure, show, save
from bokeh.io import output_file
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, TableColumn

import holoviews as hv
from holoviews import opts, dim

import panel as pn

hv.extension('bokeh')

## Create the Scatter Plot with All Indices

In [ ]:
#Create a DataFrame Just with the Last Period Metrics for Plotting the Scatter plot
##Reduce JDK_RS_ratio to 1 (Last) Period
JDK_RS_ratio_1P = pd.DataFrame(JDK_RS_ratio.iloc[-1].transpose())
JDK_RS_ratio_1P = JDK_RS_ratio_1P.rename(columns= {JDK_RS_ratio_1P.columns[0]: 'JDK_RS_ratio'})

##Reduce JDK_RS_momentum to 1 (Last) Period
JDK_RS_momentum_1P = pd.DataFrame(JDK_RS_momentum.iloc[-1].transpose())
JDK_RS_momentum_1P = JDK_RS_momentum_1P.rename(columns= {JDK_RS_momentum_1P.columns[0]: 'JDK_RS_momentum'})

#Joining the 2 Dataframes
JDK_RS_1P = pd.concat([JDK_RS_ratio_1P,JDK_RS_momentum_1P], axis=1)

##Reset the Index so the Index's names are in the Scatter
JDK_RS_1P = JDK_RS_1P.reset_index() 
order = [1,2,0] # setting column's order
JDK_RS_1P = JDK_RS_1P[[JDK_RS_1P.columns[i] for i in order]]

##Create a New Column with the Quadrants Indication
JDK_RS_1P['Quadrant'] = JDK_RS_1P['index']
for row in JDK_RS_1P['Quadrant'].index:
    if JDK_RS_1P['JDK_RS_ratio'][row] > 100 and JDK_RS_1P['JDK_RS_momentum'][row] > 100:
        JDK_RS_1P['Quadrant'][row] = 'Leading'
    elif JDK_RS_1P['JDK_RS_ratio'][row] > 100 and JDK_RS_1P['JDK_RS_momentum'][row] < 100:
        JDK_RS_1P['Quadrant'][row] = 'Lagging'
    elif JDK_RS_1P['JDK_RS_ratio'][row] < 100 and JDK_RS_1P['JDK_RS_momentum'][row] < 100:
        JDK_RS_1P['Quadrant'][row] = 'Weakening'
    elif JDK_RS_1P['JDK_RS_ratio'][row] < 100 and JDK_RS_1P['JDK_RS_momentum'][row] > 100:
        JDK_RS_1P['Quadrant'][row] = 'Improving'

In [ ]:
#Scatter Plot
scatter = hv.Scatter(JDK_RS_1P, kdims = ['JDK_RS_ratio', 'JDK_RS_momentum'])

##Colors
explicit_mapping = {'Leading': 'green', 'Lagging': 'yellow', 'Weakening': 'red', 'Improving': 'blue'}

##Defining the Charts's Area
x_max_distance = max(abs(int(JDK_RS_1P['JDK_RS_ratio'].min())-100), int(JDK_RS_1P['JDK_RS_ratio'].max())-100,
                    abs(int(JDK_RS_1P['JDK_RS_momentum'].min())-100), int(JDK_RS_1P['JDK_RS_momentum'].max())-100)
x_y_range = (100 - 1 - x_max_distance, 100 + 1 + x_max_distance)

##Plot Joining all together
scatter = scatter.opts(opts.Scatter(tools=['hover'], height = 500, width=500, size = 10, xlim = x_y_range, ylim = x_y_range,
                                   color = 'Quadrant', cmap=explicit_mapping, legend_position = 'top'))

##Vertical and Horizontal Lines
vline = hv.VLine(100).opts(color = 'black', line_width = 1)
hline = hv.HLine(100).opts(color = 'black', line_width = 1)

#All Together

full_scatter = scatter * vline * hline

In [ ]:
#Let's use the Panel library to be able to save the Table generated
p = pn.panel(full_scatter)
p.save('ScatterPlot_1Period.html') 

## Multiple Period

In [ ]:
#For multiple period we need to create a DataFrame with 3-dimensions 
    #-> to do this we create a dictionary and include each DataFrame with the assigned dictionary key being the Index
    
indices =  RRG_df.columns.unique()

multi_df = dict()
for index in indices:
    #For each of the Index will do the following procedure
    
    chosen_columns = []
    #This loop is to filter each variable's varlue in the big-dataframe and create a create a single Dataframe
    for column in RRG_df[index].columns:
        chosen_columns.append(RRG_df[index][column])
    joint_table = pd.concat(chosen_columns, axis=1)
    
    #Change the DataFrame's Header
    new_header = joint_table.iloc[0] 
    joint_table = joint_table[1:] 
    joint_table.columns = new_header
    joint_table = joint_table.loc[:,~joint_table.columns.duplicated()]
    
    #Remove the first 3 entries
    joint_table = joint_table[2:]
    
    #Create a column for the Index
    joint_table['index'] = index
    
    ##Reset the Index so the Datess are observable the Scatter
    joint_table = joint_table.reset_index()
    order = [1,2,3,0] # setting column's order
    joint_table = joint_table[[joint_table.columns[i] for i in order]]
    joint_table = joint_table.rename(columns={"level_0": "Date"})
    joint_table['Date'] = joint_table['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    ##Create a New Column with the Quadrants Indication
    joint_table['Quadrant'] = joint_table['index']
    for row in joint_table['Quadrant'].index:
        if joint_table['JDK_RS_ratio'][row] >= 100 and joint_table['JDK_RS_momentum'][row] >= 100:
            joint_table['Quadrant'][row] = 'Leading'
        elif joint_table['JDK_RS_ratio'][row] >= 100 and joint_table['JDK_RS_momentum'][row] <= 100:
            joint_table['Quadrant'][row] = 'Lagging'
        elif joint_table['JDK_RS_ratio'][row] <= 100 and joint_table['JDK_RS_momentum'][row] <= 100:
            joint_table['Quadrant'][row] = 'Weakening'
        elif joint_table['JDK_RS_ratio'][row] <= 100 and joint_table['JDK_RS_momentum'][row] >= 100:
            joint_table['Quadrant'][row] = 'Improving'
            
    #Joining the obtained Single Dataframes into the Dicitonary
    multi_df.update({index: joint_table})   

In [ ]:
#Defining the Charts's Area
x_y_max = []
for Index in multi_df.keys():
    x_y_max_ = max(abs(int(multi_df[Index]['JDK_RS_ratio'].min())-100), int(multi_df[Index]['JDK_RS_ratio'].max())-100,
                    abs(int(multi_df[Index]['JDK_RS_momentum'].min())-100), int(multi_df[Index]['JDK_RS_momentum'].max())-100)
    x_y_max.append(x_y_max_)
    
x_range = (100 - 1 - max(x_y_max), 100 + 1 + max(x_y_max))
y_range = (100 - 1 - max(x_y_max), 100 + 1.25 + max(x_y_max))
#Note: y_range has .25 extra on top because legend stays on top and option "legend_position" doesn't exist for Overlay graphs

#Include Dropdown List
def load_indices(Index): 
    scatter = hv.Scatter(multi_df[Index], kdims = ['JDK_RS_ratio', 'JDK_RS_momentum'])
    
    ##Colors
    explicit_mapping = {'Leading': 'green', 'Lagging': 'yellow', 'Weakening': 'red', 'Improving': 'blue'}
    ##Plot Joining all together
    scatter = scatter.opts(opts.Scatter(tools=['hover'], height = 500, width=500, size = 10, xlim = x_range, ylim = y_range,
                                        color = 'Quadrant', cmap=explicit_mapping,
                                       ))
    
    ##Line connecting the dots
    curve = hv.Curve(multi_df[Index], kdims = ['JDK_RS_ratio', 'JDK_RS_momentum'])
    curve = curve.opts(opts.Curve(color = 'black', line_width = 1))

    ##Vertical and Horizontal Lines
    vline = hv.VLine(100).opts(color = 'black', line_width = 1)
    hline = hv.HLine(100).opts(color = 'black', line_width = 1)    


    #All Together

    full_scatter = scatter * vline * hline * curve
    full_scatter = full_scatter.opts(legend_cols= True)

    return full_scatter
        
indices_name = RRG_df.columns.drop_duplicates().tolist()

#Instantiation the Dynamic Map object
dmap = hv.DynamicMap(load_indices, kdims='Index').redim.values(Index=indices_name)

In [ ]:
#Let's use the Panel library to be able to save the Table generated
p = pn.panel(dmap)
p.save('ScatterPlot_Multiple_Period.html', embed = True) 

## Save the Indices table to html format

In [ ]:
#Readjust the Table to be Included in Article - not to have all the columns
table = indices_data
table = table.drop(columns = ['Currency', 'Yahoo Currency Symbol'])

#Create the elements for the Table
source = ColumnDataSource(table)

columns = [TableColumn(field=Ci, title=Ci, width=20) for Ci in table.columns]

#Instantiate the Table object
data_table = DataTable(source=source, columns=columns, width=800, height=280, selectable = True, index_position = None)

#Save the Table using Bokeh library (not Panel, since Holoviews is not used here)
output_file('Indices_Table.html')
save(data_table)